
# Demonstration of the Hybrid Kernelised Expaction Maximisation (HKEM) reconstruction with SIRF
This demonstration shows how to use HKEM and investigate the role of each kernel parameter in edge preservation and noise suppression.


Authors: Daniel Deidda, Kris Thielemans and Evgueni Ovtchinnikov  
First version: 30th of September 2019  
Second Version: 30th of September 2019

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2019  National Physical Laboratory
Copyright 2019  Rutherford Appleton Laboratory STFC  
Copyright 2019  University College London.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
import string
#import scipy
#from scipy import optimize
import sirf.STIR as pet
from sirf.Utilities import examples_data_path
#os.getcwd()

In [ ]:
#%% some handy function definitions
def imshow(image, limits, title=''):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap = plt.imshow(image)
    if len(limits)==0:
        limits = [image.min(), image.max()]
                
    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

def make_positive(image_array):
    """truncate any negatives to zero"""
    image_array[image_array<0] = 0
    return image_array

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV"""
    filter = pet.TruncateToCylinderProcessor()
    filter.apply(image)

In [ ]:
#%% go to directory with input files
# adapt this path to your situation (or start everything in the relevant directory)
os.chdir(examples_data_path('PET'))

In [ ]:
#%% copy files to working folder and change directory to where the output files are
shutil.rmtree('working_folder/thorax_single_slice',True)
shutil.copytree('thorax_single_slice','working_folder/thorax_single_slice')
os.chdir('working_folder/thorax_single_slice')

## We will first create some simulated data from ground-truth images



In [ ]:
#%% Read in images
image = pet.ImageData('emission.hv')
image_array = image.as_array()*.05
image.fill(image_array)
mu_map = pet.ImageData('attenuation.hv')
mu_map_array = mu_map.as_array()

In [ ]:
#%% bitmap display of images
im_slice = image_array.shape[0]//2
plt.figure()
plt.subplot(1,2,1)
imshow(image_array[im_slice,:,:,], [], 'emission image');
plt.subplot(1,2,2)
imshow(mu_map_array[im_slice,:,:,], [], 'attenuation image');

In [ ]:
#%% save max for future displays
cmax = image_array.max()*.6

In [ ]:
#%% create acquisition model
am = pet.AcquisitionModelUsingRayTracingMatrix()
# we will increate the number of rays used for every Line-of-Response (LOR) as an example
# (it is not required for the exercise of course)
am.set_num_tangential_LORs(5)
templ = pet.AcquisitionData('template_sinogram.hs')
am.set_up(templ,image);

In [ ]:
#%% simulate some data using forward projection
acquired_data=am.forward(image)
acquisition_array = acquired_data.as_array()

In [ ]:
#%% Generate a noisy realisation of the data
noisy_array=numpy.random.poisson(acquisition_array).astype('float64')
print(' Maximum counts in the data: %d' % noisy_array.max())
# stuff into a new AcquisitionData object
noisy_data = acquired_data.clone()
noisy_data.fill(noisy_array);

In [ ]:
#%% Display bitmaps of the middle sinogram
plt.figure()
plt.subplot(1,2,1)
imshow(acquisition_array[0,im_slice,:,:,], [0,acquisition_array.max()], 'original')
plt.subplot(1,2,2)
imshow(noisy_array[0,im_slice,:,:,], [0,acquisition_array.max()], 'noisy');

# Reconstruction via a SIRF reconstruction class
While you can write your own reconstruction algorithm by using `AcquisitionModel` etc, we also
provide a few reconstruction clases. We will show how to use these here.

## create the objective function

In PET, the iterative algorithms in SIRF rely on an objective function (i.e. the function to maximise).
In this case, this is the Poisson log-likelihood (without any priors)

In [ ]:
#%% create objective function
obj_fun = pet.make_Poisson_loglikelihood(noisy_data)
# We could set acquisition model but the default (ray-tracing) is in this case ok
# obj_fun.set_acquisition_model(am)
# we could also add a prior, but we will not do that here (although the rest of the exercise would still work)
#obj_fun.set_prior(prior)

##  create KOSMAPOSL reconstructor
This implements the Ordered Subsets HKEM
In this section we define all parameter

In [ ]:
recon = pet.KOSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)

recon.set_input(noisy_data)

# set up anatomical image
anatomical_image = mu_map.clone()
anatomical_image_array = anatomical_image.as_array()
anatomical_image_array[anatomical_image_array < 0] = 0
anatomical_image.fill(image_array)

recon.set_anatomical_prior(anatomical_image)
recon.set_only_2D(True)
recon.set_num_non_zero_features(1)

recon.set_num_subsets(4)
num_iters=40
recon.set_num_subiterations(num_iters)

## Use this reconstructor!

## Study parameter "neighbourhood size", n

In [ ]:
#%%  create initial image
# we could just use a uniform image but here we will create a disk with a different
# initial value (this will help the display later on)
init_image=image.clone()
init_image.fill(cmax / 4)
make_cylindrical_FOV(init_image)
# display
idata = init_image.as_array()
im_slice = idata.shape[0] // 2
#plt.figure()
#imshow(idata[im_slice,:,:],[0,cmax], 'initial image');

In [ ]:
#%% reconstruct the image 
H1n_reconstructed_image = [] 
H0n_reconstructed_image = []

#fix other parameters
recon.set_sigma_m(0.5)
recon.set_sigma_p(0.5)
recon.set_sigma_dm(3.0)
recon.set_sigma_dp(3.0)

n={3, 5, 7, 9}
ii=0
for i in n:
    
    H1n_reconstructed_image.append(init_image.clone())
    H0n_reconstructed_image.append(init_image.clone())
    
    recon.set_num_neighbours(i)
    j="{}".format(i)
    jj=j.replace(".", "_")
    
#   reconstruct using side information from MR and PET (H0 is KEM H1 is HKEM)
    output_prefix = "H1_N{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(True)
    recon.set_up(H1n_reconstructed_image[ii])
    recon.reconstruct(H1n_reconstructed_image[ii])

#   reconstruct using only side information from MR (H0 is KEM H1 is HKEM)
    output_prefix = "H0_N{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(False)
    recon.set_up(H0n_reconstructed_image[ii])
    recon.reconstruct(H0n_reconstructed_image[ii])
    ii=ii+1;

In [ ]:
#%% bitmap display of images
# define lists

H1n_reconstructed_array = []
H0n_reconstructed_array = []
diffn_image_array = []
H1n_error_array = []
H0n_error_array = []
ii=0

for i in n:
    
    j="{}".format(i)
    jj=j.replace(".", "_")
    #H0n_reconstructed_image[ii]=pet.ImageData('H0_N'+jj+'_12.hv')
    H1n_reconstructed_array.append(H1n_reconstructed_image[ii].as_array())
    H0n_reconstructed_array.append(H0n_reconstructed_image[ii].as_array())
    
#   anatomical_image_array = anatomical_image.as_array()
    diffn_image_array.append(H1n_reconstructed_array[ii] - H0n_reconstructed_array[ii])
    H1n_error_array.append(image_array - H1n_reconstructed_array[ii])
    H0n_error_array.append(image_array - H0n_reconstructed_array[ii])

    plt.figure()
    plt.subplot(3,3,1)
    imshow(image_array[im_slice,:,:,], [0,cmax*1.5],'True image')
    plt.subplot(3,3,2)
    imshow(H1n_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'HKEM, n ='+j)
    plt.subplot(3,3,3)
    imshow(H0n_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'KEM, n ='+j)
    plt.subplot(3,3,4)
    imshow(diffn_image_array[ii][im_slice,:,:,], [-cmax*0.1,cmax*0.1], 'HKEM - KEM');
    plt.subplot(3,3,5)
    imshow(H1n_error_array[ii][im_slice,:,:,], [-cmax*1,cmax*1], 'HKEM error')
    plt.subplot(3,3,6)
    imshow(H0n_error_array[ii][im_slice,:,:,], [-cmax*1,cmax*.1], 'KEM error')

    ii=ii+1;

## Study parameter sigma_m (MR edge preservation)

In [ ]:
#%% reconstruct the image 
H1m_reconstructed_image = [] 
H0m_reconstructed_image = []

#fix other parameters
recon.set_num_neighbours(5)
recon.set_sigma_p(0.5)
recon.set_sigma_dm(3.0)
recon.set_sigma_dp(3.0)

sigma_m={0.05, 0.1, 0.5, 1, 3, 5, 8, 10}
ii=0
for i in sigma_m:
    
    H1m_reconstructed_image.append(init_image.clone())
    H0m_reconstructed_image.append(init_image.clone())
    
    recon.set_sigma_m(i)
    j="{}".format(i)
    jj=j.replace(".", "_")
    
#   reconstruct using side information from MR and PET (H0 is KEM H1 is HKEM)
    output_prefix = "H1_N5_M{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(True)
    recon.set_up(H1m_reconstructed_image[ii])
    recon.reconstruct(H1m_reconstructed_image[ii])

#   reconstruct using only side information from MR (H0 is KEM H1 is HKEM)
    output_prefix = "H0_N5_M{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(False)
    recon.set_up(H0m_reconstructed_image[ii])
    recon.reconstruct(H0m_reconstructed_image[ii])
    ii=ii+1;

In [ ]:
#%% bitmap display of images
# define lists
H1m_reconstructed_array = []
H0m_reconstructed_array = []
diffm_image_array = []
H1m_error_array = []
H0m_error_array = []
ii=0
sigma_m={0.05, 0.1, 0.5, 1, 3, 5, 8, 10}
for i in sigma_m:
    
    j="{}".format(i)
    jj=j.replace(".", "_")
    
    H1m_reconstructed_array.append(H1m_reconstructed_image[ii].as_array())
    H0m_reconstructed_array.append(H0m_reconstructed_image[ii].as_array())
    
#   anatomical_image_array = anatomical_image.as_array()
    diffm_image_array.append(H1m_reconstructed_array[ii] - H0m_reconstructed_array[ii])
    H1m_error_array.append(image_array - H1m_reconstructed_array[ii])
    H0m_error_array.append(image_array - H0m_reconstructed_array[ii])

    plt.figure()
    plt.subplot(3,3,1)
    imshow(image_array[im_slice,:,:,], [0,cmax*1.5],'True image')
    plt.subplot(3,3,2)
    imshow(H1m_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'HKEM, sigma_m='+j)
    plt.subplot(3,3,3)
    imshow(H0m_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'KEM, sigma_m='+j)
    plt.subplot(3,3,4)
    imshow(diffm_image_array[ii][im_slice,:,:,], [-cmax*0.1,cmax*0.1], 'HKEM - KEM');
    plt.subplot(3,3,5)
    imshow(H1m_error_array[ii][im_slice,:,:,], [-cmax*1,cmax*1], 'HKEM error')
    plt.subplot(3,3,6)
    imshow(H0m_error_array[ii][im_slice,:,:,], [-cmax*1,cmax*1], 'KEM error')

    ii=ii+1;

## Study parameter sigma_p (PET edge preservation)


In [ ]:
#%% reconstruct the image 
H1p_reconstructed_image = [] 
H0p_reconstructed_image = []

#fix other parameters
recon.set_num_neighbours(5)
recon.set_sigma_m(0.5)
recon.set_sigma_dm(3.0)
recon.set_sigma_dp(3.0)

sigma_p={0.05, 0.1, 0.5, 1, 3, 5, 8, 10}
ii=0
for i in sigma_p:
    
    H1p_reconstructed_image.append(init_image.clone())
    H0p_reconstructed_image.append(init_image.clone())
    
    recon.set_sigma_p(i)
    j="{}".format(i)
    jj=j.replace(".", "_")
    
#   reconstruct using side information from MR and PET (H0 is KEM H1 is HKEM)
    output_prefix = "H1_N5_P{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(True)
    recon.set_up(H1p_reconstructed_image[ii])
    recon.reconstruct(H1p_reconstructed_image[ii])

#   reconstruct using only side information from MR (H0 is KEM H1 is HKEM)
    output_prefix = "H0_N5_P{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(False)
    recon.set_up(H0p_reconstructed_image[ii])
    recon.reconstruct(H0p_reconstructed_image[ii])
    ii=ii+1;

In [ ]:
#%% bitmap display of images
# define lists
H1p_reconstructed_array = []
H0p_reconstructed_array = []
diffp_image_array = []
H1p_error_array = []
H0p_error_array = []
ii=0

for i in sigma_p:
    
    j="{}".format(i)
    jj=j.replace(".", "_")
    
    H1p_reconstructed_array.append(H1p_reconstructed_image[ii].as_array())
    H0p_reconstructed_array.append(H0p_reconstructed_image[ii].as_array())
    
#   anatomical_image_array = anatomical_image.as_array()
    diffp_image_array.append(H1p_reconstructed_array[ii] - H0p_reconstructed_array[ii])
    H1p_error_array.append(image_array - H1p_reconstructed_array[ii])
    H0p_error_array.append(image_array - H0p_reconstructed_array[ii])

    plt.figure()
    plt.subplot(3,3,1)
    imshow(image_array[im_slice,:,:,], [0,cmax*1.5],'True image')
    plt.subplot(3,3,2)
    imshow(H1p_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'HKEM, sigma_p='+j)
    plt.subplot(3,3,3)
    imshow(H0p_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'KEM, sigma_p='+j)
    plt.subplot(3,3,4)
    imshow(diffp_image_array[ii][im_slice,:,:,], [-cmax*0.1,cmax*0.1], 'HKEM - KEM');
    plt.subplot(3,3,5)
    imshow(H1p_error_array[ii][im_slice,:,:,], [-cmax*1.,cmax*1.], 'HKEM error')
    plt.subplot(3,3,6)
    imshow(H0p_error_array[ii][im_slice,:,:,], [-cmax*1.,cmax*1.], 'KEM error')

    ii=ii+1;

## Study parameter sigma_d (smoothing)


In [ ]:
#%% reconstruct the image 
H1d_reconstructed_image = [] 
H0d_reconstructed_image = []

#fix other parameters
recon.set_num_neighbours(5)
recon.set_sigma_m(0.5)
recon.set_sigma_p(0.5)

sigma_dm={0.05, 0.1, 0.5, 1, 3, 5, 8, 10}
ii=0
for i in sigma_dm:
    
    H1d_reconstructed_image.append(init_image.clone())
    H0d_reconstructed_image.append(init_image.clone())
    
    recon.set_sigma_dp(i)
    recon.set_sigma_dm(i)
    
    j="{}".format(i)
    jj=j.replace(".", "_")
    
#   reconstruct using side information from MR and PET (H0 is KEM H1 is HKEM)
    output_prefix = "H1_N5_D{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(True)
    recon.set_up(H1d_reconstructed_image[ii])
    recon.reconstruct(H1d_reconstructed_image[ii])

#   reconstruct using only side information from MR (H0 is KEM H1 is HKEM)
    output_prefix = "H0_N5_D{}".format(jj)
#   set up the reconstructor
    recon.set_output_filename_prefix(output_prefix)
    recon.set_hybrid(False)
    recon.set_up(H0d_reconstructed_image[ii])
    recon.reconstruct(H0d_reconstructed_image[ii])
    ii=ii+1;

In [ ]:
#%% bitmap display of images
# define lists
H1d_reconstructed_array = []
H0d_reconstructed_array = []
diffd_image_array = []
H1d_error_array = []
H0d_error_array = []
ii=0

for i in sigma_dm:
    
    j="{}".format(i)
    jj=j.replace(".", "_")
    
    H1d_reconstructed_array.append(H1d_reconstructed_image[ii].as_array())
    H0d_reconstructed_array.append(H0d_reconstructed_image[ii].as_array())
    
#   anatomical_image_array = anatomical_image.as_array()
    diffd_image_array.append(H1d_reconstructed_array[ii] - H0d_reconstructed_array[ii])
    H1d_error_array.append(image_array - H1d_reconstructed_array[ii])
    H0d_error_array.append(image_array - H0d_reconstructed_array[ii])

    plt.figure()
    plt.subplot(3,3,1)
    imshow(image_array[im_slice,:,:,], [0,cmax*1.5],'True image')
    plt.subplot(3,3,2)
    imshow(H1d_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'HKEM, sigma_d='+j)
    plt.subplot(3,3,3)
    imshow(H0d_reconstructed_array[ii][im_slice,:,:,], [0,cmax*1.5], 'KEM, sigma_d='+j)
    plt.subplot(3,3,4)
    imshow(diffd_image_array[ii][im_slice,:,:,], [-cmax*0.1,cmax*0.1], 'HKEM - KEM');
    plt.subplot(3,3,5)
    imshow(H1d_error_array[ii][im_slice,:,:,], [-cmax*1.,cmax*1.], 'HKEM error')
    plt.subplot(3,3,6)
    imshow(H0d_error_array[ii][im_slice,:,:,], [-cmax*1.,cmax*1.], 'KEM error')
    
    ii=ii+1;
    